In [3]:
import pandas as pd

In [4]:
df = pd.read_excel('RBB_PO.xlsx',skiprows=3)

df_cleaned = df.dropna(axis=1,how='all')
df_cleaned

,Unnamed: 1,Unnamed: 2
0,2.0CT HOOP EARRING (ER0000840K),NaN
1,SIZE,~
2,THICKNESS,~
3,METAL,14KT
4,METAL COLOR,WHITE GOLD
5,WEIGHT WHEN FINISHED,3.85GM
6,STONE WEIGHT,2.2MM (0.07 X 30 STONES) - 2.00CTW MIN
7,STONE SIZE,2.2mm Each Stone
8,STAMP DETAILS,RB Logo / 14KT /
9,STAMP LOCATION,Inner PART


In [5]:
# Build structured dataframe from df_cleaned as per requirements
import re
from datetime import datetime

# Ensure we have two columns: key and value
key_col = df_cleaned.columns[0]
val_col = df_cleaned.columns[1] if len(df_cleaned.columns) > 1 else None

# Create a simple key->value mapping (first non-null value per key)
meta_rows = df_cleaned[[key_col] + ([val_col] if val_col is not None else [])].copy()
meta_rows.columns = ["Key", "Value"] if val_col is not None else ["Key"]

# Helper to fetch a value by (case-insensitive) key prefix
def get_value(prefix: str, default: str = "") -> str:
    if val_col is None:
        return default
    mask = meta_rows["Key"].astype(str).str.strip().str.upper().str.startswith(prefix.upper())
    vals = meta_rows.loc[mask, "Value"].dropna().astype(str).str.strip()
    return vals.iloc[0] if not vals.empty else default

# Title row (first row, first column)
first_cell = str(df_cleaned.iloc[0, 0]) if not df_cleaned.empty else ""

# Extract CT from title like "2.0CT ..." -> formatted as "2.00 CT"
ct_match = re.search(r"(\d+(?:\.\d+)?)\s*CT", first_cell, flags=re.IGNORECASE)
ct_value = None
if ct_match:
    try:
        ct_numeric = float(ct_match.group(1))
        ct_value = f"{ct_numeric:.2f} CT"
    except ValueError:
        ct_value = ct_match.group(1).strip() + " CT"
# Fallback: try from STONE WEIGHT if title did not contain CT
if not ct_value:
    stone_weight = get_value("STONE WEIGHT", "")
    ct_match = re.search(r"(\d+(?:\.\d+)?)\s*CT", str(stone_weight), flags=re.IGNORECASE)
    if ct_match:
        try:
            ct_numeric = float(ct_match.group(1))
            ct_value = f"{ct_numeric:.2f} CT"
        except ValueError:
            ct_value = ct_match.group(1).strip() + " CT"

# StyleCode from STYLE # row or from parentheses in the title
style_code = get_value("STYLE #", "").strip()
if not style_code:
    paren = re.search(r"\(([^)]+)\)", first_cell)
    if paren:
        style_code = paren.group(1).strip()

# Size (may be absent or placeholder like ~)
raw_size = get_value("SIZE", "").strip()
item_size = "" if raw_size in {"", "~", "-", "NA", "NaN"} else raw_size

# OrderQty (extract leading integer from strings like "100 PAIRS")
order_qty_str = get_value("ORDER QTY", "")
order_qty_match = re.search(r"\d+", str(order_qty_str))
order_qty = int(order_qty_match.group(0)) if order_qty_match else None

# PO DATE -> format to dd-mm-YYYY and embed in phrase
po_date_raw = get_value("PO DATE", "")
po_date_fmt = ""
if po_date_raw:
    try:
        # Pandas can parse many formats; fallback to datetime if needed
        po_dt = pd.to_datetime(po_date_raw)
        po_date_fmt = po_dt.strftime("%d-%m-%Y")
    except Exception:
        # Try manual splits (YYYY-MM-DD)
        try:
            parts = str(po_date_raw).split(" ")[0].split("-")
            if len(parts) == 3:
                y, m, d = parts
                po_date_fmt = f"{d}-{m}-{y}"
        except Exception:
            po_date_fmt = str(po_date_raw)
item_po_no = f"email dated as on {po_date_fmt}" if po_date_fmt else ""

# Metal mapping: combine METAL (karat) + METAL COLOR into code like G14W
metal_karat_raw = get_value("METAL", "").upper().replace("KARAT", "KT").strip()
metal_color_raw = get_value("METAL COLOR", "").upper().strip()

# Extract karat number (e.g., 14 from "14KT")
karat_num_match = re.search(r"(8|9|10|14|18|22|24)\s*KT", metal_karat_raw)
karat_num = karat_num_match.group(1) if karat_num_match else ""

# Tone mapping from color
color_to_tone = {
    "WHITE": "W",
    "WHITE GOLD": "W",
    "YELLOW": "Y",
    "YELLOW GOLD": "Y",
    "ROSE": "R",
    "ROSE GOLD": "R",
    "PINK": "R",
}

def normalize_color(color: str) -> str:
    color = color.upper()
    # Look for keywords to determine tone
    if "WHITE" in color:
        return "W"
    if "YELLOW" in color:
        return "Y"
    if "ROSE" in color or "PINK" in color:
        return "R"
    return ""

tone_char = normalize_color(metal_color_raw)

# Metal code like G14W if we have karat and tone
metal_code = f"G{karat_num}{tone_char}" if karat_num and tone_char else ""

# Tone (lowercase single letter per requirement)
tone_value = tone_char.lower() if tone_char else ""

# Special Remarks: "End Customer Name, 14k WHITE GOLD, SZ-<size>" (size only if present)
end_customer_name = input("Enter End Customer Name: ").strip()
priority_value = input("Enter Priority: ").strip()

size_fragment = f", SZ-{item_size}" if item_size else ""
special_remarks = ""
if end_customer_name or karat_num or metal_color_raw:
    karat_k = (karat_num + "k") if karat_num else ""
    special_remarks = f"{end_customer_name}, {karat_k} {metal_color_raw}{size_fragment}".strip().strip(", ")

# Design Production Instruction: White Rodium if tone is W else No Rodium
design_prod_instr = "White Rodium" if tone_char == "W" else "No Rodium"

# Stamp Instruction: "14k +RB LOGO+2.00 CT"
karat_k = (karat_num + "k") if karat_num else ""
stamp_instruction = f"{karat_k} +RB LOGO+{ct_value}".strip() if karat_k or ct_value else "RB LOGO"

# Assemble final dataframe (single row for this item)
columns = [
    "SrNo",
    "StyleCode",
    "ItemSize",
    "OrderQty",
    "OrderItemPcs",
    "Metal",
    "Tone",
    "ItemPoNo",
    "ItemRefNo",
    "StockType",
    "Priority",
    "MakeType",
    "CustomerProductionInstruction",
    "SpecialRemarks",
    "DesignProductionInstruction",
    "StampInstruction",
    "OrderGroup",
    "SKUNo",
    "Basestoneminwt",
    "Basestonemaxwt",
    "Basemetalminwt",
    "Basemetalmaxwt",
    "Productiondeliverydate",
    "Expecteddeliverydate",
    "Blank",
    "SetPrice",
    "StoneQuality",
]

row = {
    "SrNo": 1,
    "StyleCode": style_code,
    "ItemSize": item_size,
    "OrderQty": order_qty,
    "OrderItemPcs": "",
    "Metal": metal_code,
    "Tone": tone_value,
    "ItemPoNo": item_po_no,
    "ItemRefNo": "",
    "StockType": "",
    "Priority": priority_value,
    "MakeType": "",
    "CustomerProductionInstruction": "",
    "SpecialRemarks": special_remarks,
    "DesignProductionInstruction": design_prod_instr,
    "StampInstruction": stamp_instruction,
    "OrderGroup": end_customer_name,
    "SKUNo": "",
    "Basestoneminwt": "",
    "Basestonemaxwt": "",
    "Basemetalminwt": "",
    "Basemetalmaxwt": "",
    "Productiondeliverydate": "",
    "Expecteddeliverydate": "",
    "Blank": "",
    "SetPrice": "",
    "StoneQuality": "",
}

final_df = pd.DataFrame([row], columns=columns)
final_df


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,Blank,SetPrice,StoneQuality
0,1,ER0000840K,,100,,G14W,w,email dated as on 03-10-2024,,,...,,,,,,,,,,


In [7]:
final_df.to_excel('RBL.xlsx')